In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import logistic
from scipy.optimize import minimize
from scipy.special import sech

# Parameters and sample size
N = 10**5  # Number of samples
lambda_param = 1  # Parameter value for the CDF and PDF

# Step 1: Generate uniform random probabilities and transform them
p_k = np.random.uniform(0, 1, N)
x_k = np.log(p_k / (1 - p_k))  # Apply inverse CDF transformation for lambda=1

# Plot histogram of x_k
plt.hist(x_k, bins=100, density=True, alpha=0.6, color='skyblue', edgecolor='black', label="Generated Sample Density")

# Step 2: Define the PDF and overlay it
def logistic_pdf(x, lambda_param=1):
    return (lambda_param / 4) * sech(lambda_param * x / 2)**2

x_vals = np.linspace(-10, 10, 1000)
pdf_vals = logistic_pdf(x_vals, lambda_param)

# Plot the theoretical PDF on top of the histogram
plt.plot(x_vals, pdf_vals, color='red', lw=2, label="Theoretical PDF $f_x(x; \\lambda=1)$")
plt.xlabel("x")
plt.ylabel("Density")
plt.title("Histogram of Generated Sample with Theoretical PDF Overlay")
plt.legend()
plt.show()

# Step 3: Maximum Likelihood Estimation (MLE) for lambda
def neg_log_likelihood(lambda_val, x_k):
    if lambda_val <= 0:  # To ensure positive lambda
        return np.inf
    pdf_vals = (lambda_val / 4) * sech(lambda_val * x_k / 2)**2
    return -np.sum(np.log(pdf_vals))

# Minimize negative log-likelihood
result = minimize(neg_log_likelihood, x0=1, args=(x_k), method='L-BFGS-B', bounds=[(0.01, None)])
lambda_hat = result.x[0]
print(f"Estimated λ (MLE): {lambda_hat}")

# Step 4: Estimate uncertainty in lambda using graphical method
lambda_range = np.linspace(lambda_hat - 0.5, lambda_hat + 0.5, 100)
nll_vals = [neg_log_likelihood(lam, x_k) for lam in lambda_range]

# Find the uncertainty where NLL increases by 0.5 from minimum
min_nll = min(nll_vals)
lambda_plus = lambda_range[np.where(nll_vals >= min_nll + 0.5)[0][0]]
lambda_minus = lambda_range[np.where(nll_vals >= min_nll + 0.5)[0][-1]]
lambda_uncertainty = (lambda_plus - lambda_minus) / 2
print(f"λ ± Uncertainty: {lambda_hat} ± {lambda_uncertainty}")

# Step 5: Generate logistic sample using scipy and compare
scipy_sample = logistic.rvs(size=N)

# Plot residuals histogram
bins = np.linspace(-10, 10, 100)
hist_my_sample, _ = np.histogram(x_k, bins=bins, density=True)
hist_scipy_sample, _ = np.histogram(scipy_sample, bins=bins, density=True)
bin_centers = (bins[1:] + bins[:-1]) / 2
residuals = (hist_my_sample - hist_scipy_sample) / np.sqrt(hist_my_sample + 1e-6)  # Approximate uncertainty

plt.bar(bin_centers, residuals, width=0.2, color='purple', alpha=0.6, label="Residuals")
plt.xlabel("x")
plt.ylabel("Residual")
plt.title("Residuals Histogram between Generated Sample and Scipy Logistic Sample")
plt.legend()
plt.show()

# Step 6: Standardize x_k to create z_k and estimate pi using variance
mu_x = np.mean(x_k)
var_x = np.var(x_k)
z_k = (x_k - mu_x) / np.sqrt(3 * var_x)

# Repeat MLE for the rescaled variable z_k
result_z = minimize(neg_log_likelihood, x0=1, args=(z_k), method='L-BFGS-B', bounds=[(0.01, None)])
lambda_hat_z = result_z.x[0]
print(f"Estimated λ for z (MLE): {lambda_hat_z}")

# Graphical uncertainty estimation for λ_hat_z
nll_vals_z = [neg_log_likelihood(lam, z_k) for lam in lambda_range]
min_nll_z = min(nll_vals_z)
lambda_plus_z = lambda_range[np.where(nll_vals_z >= min_nll_z + 0.5)[0][0]]
lambda_minus_z = lambda_range[np.where(nll_vals_z >= min_nll_z + 0.5)[0][-1]]
lambda_uncertainty_z = (lambda_plus_z - lambda_minus_z) / 2
print(f"λ for z ± Uncertainty: {lambda_hat_z} ± {lambda_uncertainty_z}")

# Final conclusion on compatibility with π
print(f"Estimated π-compatible λ for z: {lambda_hat_z} ± {lambda_uncertainty_z}")
